## Measuring energy and energy variance with classical shadows

Related Paper [Huang et al, Nat Phys 2020](https://doi.org/10.1038/s41567-020-0932-7)

In [2]:
using Revise
using RandomMeas
using Statistics
using ProgressMeter

# Ising Hamiltonian

We first construct the Hamiltonian of the transverse field Ising model with N qubits.

In [3]:
N = 6 # Number of qubits
ξ = siteinds("Qubit", N) # ITensor site indices for classical shadows
# Define the Hamiltonian
J = -1. # Ising interaction strength
B = 1. # Transverse field strength
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising XX interactions
  ampo .+= J , "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field Z
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ) # Hamiltonian as MPO
H2 = apply(H,H); # H^2 for later use

# Groundstate energy and variance

First, we compute the ground state $\psi_G$ of the transverse field Ising model with DMRG.

In [4]:
# Define the DMRG parameters 
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = random_mps(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)

# Run DMRG to get the ground state
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψG = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-7.243946309928767  maxlinkdim=4 maxerr=8.88E-16 time=17.120
After sweep 2 energy=-7.296187134287553  maxlinkdim=8 maxerr=1.60E-16 time=0.017
After sweep 3 energy=-7.296229810318139  maxlinkdim=6 maxerr=1.24E-11 time=0.018
After sweep 4 energy=-7.296229810530125  maxlinkdim=6 maxerr=2.50E-12 time=0.018
After sweep 5 energy=-7.296229810530149  maxlinkdim=6 maxerr=2.50E-12 time=0.014

Ground state energy:  -7.296229810530149

---------------------------------------



We compute its energy and variance.

In [5]:
E_G = real(inner(ψG', H,ψG))
println("final energy ",E_G)
E2_G = real(inner(ψG', H2,ψG))
println("final expect of H2 ",E2_G)
Var_E_G = E2_G-E_G^2
println("final energy variance ",Var_E_G)

final energy -7.296229810530149
final expect of H2 53.234969448397685
final energy variance 3.2886759981920477e-10


### Estimation with classical shadows 

We estimate its energy and variance with classical shadows via randomized measurements.

In [6]:
NU = 1000 # Number of random unitaries
NM = 50
measurement_group = MeasurementGroup(ψG,NU,NM;mode="dense",progress_bar=true);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:54


In [7]:
factorized_shadows = get_factorized_shadows(measurement_group);

In [8]:
Es_m,Es_ste = get_expect_shadow(H,factorized_shadows,compute_sem=true)
E2s_m,E2s_ste = get_expect_shadow(H2,factorized_shadows,compute_sem=true)
Es_m = real(Es_m)
E2s_m = real(E2s_m)
Evar = E2s_m .- Es_m.^2
Evar_ste = Evar*sqrt(4*Es_ste^2*Es_m^2 + E2s_ste^2)
println("Estimated energy  ", round(Es_m,digits=2), "±", round(Es_ste, sigdigits=1))
println("Estimated expectation value of H^2  ", round(E2s_m,digits=1), "±", round(E2s_ste, sigdigits=1))
println("Estimated energy variance ", round(Evar,digits=1), "±", round(Evar_ste, sigdigits=1)) # We oversimplify here. E2s_m and E_m are not independent.

Estimated energy  -7.43±0.03
Estimated expectation value of H^2  54.6±0.4
Estimated energy variance -0.6±-0.4


# Output state of quantum circuit

We construct a state as output of a quantum circuit.

In [14]:
η = 4 # Number of layer blocks
m = 4 # Number of layers per block
# Build the gate structure
circuit = ITensor[]
for d in 1:η
    xx_layer = [op("Rxx", ξ[j], ξ[j+1], ϕ=1*d/η/m) for j in 1:(N - 1)] 
    z_layer = [op("Rz", ξ[j], θ=2*B/m) for j in 1:N] 
    for t in 1:m
        append!(circuit, xx_layer)
        append!(circuit, z_layer)
    end
end
ψ0 = MPS(ξ,["Up" for n in 1:N]);
ψt = apply(circuit,ψ0; cutoff=1E-8)
normalize!(ψt);


We compute its energy and variance.

In [15]:
Et = real(inner(ψt', H,ψt)) # Energy of ψt 
println("final energy ",Et)
Var_Et = real(inner(ψt', H2,ψt))-Et^2 # Energy variance of ψt
println("final energy variance ",Var_Et)

final energy -6.899080647830891
final energy variance 1.9151339083140613


In [19]:
NU = 1000 # Number of random unitaries
NM = 50 # Number of measurements per unitary
measurement_group = MeasurementGroup(ψt,NU,NM;mode="dense",progress_bar=true);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


In [20]:
factorized_shadows = get_factorized_shadows(measurement_group);

In [21]:
Es_m,Es_ste = get_expect_shadow(H,factorized_shadows,compute_sem=true)
E2s_m,E2s_ste = get_expect_shadow(H2,factorized_shadows,compute_sem=true)
Es_m = real(Es_m)
E2s_m = real(E2s_m)
Evar = E2s_m .- Es_m.^2
Evar_ste = Evar*sqrt(4*Es_ste^2*Es_m^2 + E2s_ste^2)
println("Estimated energy of the circuit generated state ", round(Es_m,digits=2), "±", round(Es_ste, sigdigits=1))
println("Estimated expectation value of H^2 of the circuit generated state ", round(E2s_m,digits=1), "±", round(E2s_ste, sigdigits=1))
println("Estimated energy variance of the circuit generated state ", round(Evar,digits=1), "±", round(Evar_ste, sigdigits=1)) # We oversimplify here. E2s_m and E_m are not independent.

Estimated energy of the circuit generated state -6.94±0.03
Estimated expectation value of H^2 of the circuit generated state 49.7±0.4
Estimated energy variance of the circuit generated state 1.6±1.0
